In [44]:
import csv
import cv2 as cv
import numpy as np
import os


def get_object_info(file_path, result_path, txt_path):

    def find_object(img):
        """Finds the largest object and computes the esd.
        Args:
            img (np.array): gray scale image
        Returns:
            ESD: 2 * sqrt(Area / PI)
        """
        max_val = np.mean(img) - np.std(img)

        img_c = img.copy()
        img_c[np.where(img < max_val)] = 255
        img_c[np.where(img >= max_val)] = 0
        img = img_c
        thresh = img
        # thresh = cv.threshold(img, max_val, 255, cv.THRESH_BINARY)[1]

        cnts = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)[0]
        cnt_areas = {cv.contourArea(cnt): cnt for cnt in cnts}
        if cnt_areas:
            cnt_areas_list = list(cnt_areas)
            cnt_areas_list.sort()
            max_area = cnt_areas_list[-1]
            esd = 2 * np.sqrt(max_area / np.pi)
            return esd
        else:
            return 0

    fns = os.listdir(file_path)
    result_path = result_path
    txt_path = txt_path
    info = []

    for fn in fns:
        fn = os.path.join(file_path, fn)
        fn, ending = fn[:-4], fn[-4:]
        txtfn = os.path.join(txt_path, fn[-55:])
        try:
            pos = fn.find('bar')
            try:
                depth = float(fn[pos-5:pos])
                depth = (depth-1)/0.1
            except ValueError:
                depth = float(fn[pos-4:pos])
                depth = (depth-1)/0.1
        except:
            continue
            
        #print(txtfn)
        if ending == ".jpg" or ending == ".tif" or ending == ".png":
            img = cv.imread(fn + ending)
            try:
                bounding_info = np.loadtxt(txtfn + ".txt")
                #print('found: ' + fn +'.txt')
            except:
                continue
            img_h, img_w = img.shape[:2]

            for counter, ( instclass, rel_x, rel_y, rel_w, rel_h, conf) in enumerate(bounding_info):
                c_x = round(rel_x * img_w)  #x-position of center
                c_y = round(rel_y * img_h)  #y-position of center
                w = round(rel_w * img_w)
                h = round(rel_h * img_h)
                x = c_x - w // 2            #x-position of top-left corner
                y = c_y - h // 2            #y-position of top-left corner

                crop = img[y:y+h, x:x+w]
                
                gray = cv.cvtColor(crop, cv.COLOR_BGR2GRAY)
                esd = find_object(gray)

                info.append((fn, counter, c_x, c_y, w, h, x, y, esd, instclass, depth))

    with open(result_path, "w", newline="") as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(["Filename", "ID", "Center x in px", "Center y in px", "Width in px", "Height in px", "x in px", "y in px", "ESD in px", "class", "depth in meters below surface"])
        for row in info:
            writer.writerow(row)


In [45]:
from tkinter.filedialog import askdirectory

IMG_PATH = askdirectory(title="Image Path")
TXT_PATH = askdirectory(title="Label Path")
RESULT_PATH = TXT_PATH + '/results.csv'

print('labels:' + TXT_PATH)
get_object_info(IMG_PATH, RESULT_PATH, TXT_PATH)

labels:/Users/vdausmann/pytorch-test/yolov5/runs/detect/15_03_2020_1651_M4/labels
